# Прогнозирование принадлежности района к кластеру с использованием модели логистической регрессии   
## (в развитие [этой](https://github.com/s-eremeev/eremeev/blob/eremeev/test_project_Califitnia/test_project.ipynb) работы)

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np

**neighborhoods** - случайная выборка районов штата Калифорния. Каждая строка таблицы представляет собой какой-то район, имеющий такие атрибуты, как:

    - id - уникальный идентификатор, primary key;
    - longitude - географическая долгота, град.;
    - latitude - географическая широта, град.;
    - median_house_value - медианная стоимость жилого дома в данном районе, доллар США;
    - population - численность населения района, чел.;
    - ocean_proximity - кластер по принципу близости к океану, качественная переменная;
    
#### Для некоторых районов принадлежность к кластеру не указана, её, собственно, и нужно спрогнозировать.

In [ ]:
#установление соединения с БД
engine = create_engine('sqlite:///test_project.db')
# загрузим neighborhoods в df
sql = 'select * from neighborhoods n;'
df_neighborhoods = pd.read_sql(sql, engine)
df_neighborhoods.head(20)

,id,longitude,latitude,median_house_value,population,ocean_proximity
0,0,-122.23,37.88,452600.0,322.0,None
1,1,-122.22,37.86,358500.0,2401.0,NEAR BAY
2,2,-122.24,37.85,352100.0,496.0,NEAR BAY
3,3,-122.25,37.85,341300.0,558.0,None
4,4,-122.25,37.85,342200.0,565.0,NEAR BAY
5,5,-122.25,37.85,269700.0,413.0,NEAR BAY
6,6,-122.25,37.84,299200.0,1094.0,None
7,7,-122.25,37.84,241400.0,1157.0,None
8,8,-122.26,37.84,226700.0,1206.0,NEAR BAY
9,9,-122.25,37.84,261100.0,1551.0,None


### [Логистическая регрессия или логит-модель](https://ru.wikipedia.org/wiki/Логистическая_регрессия) (англ. logit model) — статистическая модель, используемая для прогнозирования вероятности возникновения некоторого события путём его сравнения с логистической кривой. Эта регрессия выдаёт ответ в виде вероятности бинарного события (1 или 0).

### Для каждого кластера на известных данных о принадлежности района обучим модель логистической регрессии (бинарная классификация: 1 - 0, принадлежит - не принадлежит) и применим получившиеся модели к неизвестным данным.

In [ ]:
from sklearn.linear_model import LogisticRegression # импортируем модуль логистической регрессии
# разделим df_neighborhoods на два df: df_neighborhoods_ocean_proximity, где ocean_proximity значим, и df_neighborhoods_no_ocean_proximity, где он None
df_neighborhoods_ocean_proximity = df_neighborhoods[df_neighborhoods['ocean_proximity'].notna()]
df_neighborhoods_no_ocean_proximity = df_neighborhoods[df_neighborhoods['ocean_proximity'].isna()]
# сформируем список кластеров для построения моделей
clusters_list = ['<1H_OCEAN', 'INLAND', 'ISLAND', 'NEAR_BAY', 'NEAR_OCEAN']
# для каждого кластера:
for cluster in clusters_list:
    # построим и обучи модель логистической регрессии
    X, y = df_clusters[['longitude', 'latitude', 'median_house_value', 'population']], df_clusters[cluster]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    model = LogisticRegression()
    model.fit(X_train, y_train)
    # возьмём известные данные с неизвестными кластерами
    log_reg = df_neighborhoods_no_ocean_proximity[['longitude', 'latitude', 'median_house_value', 'population']]
    # и построим прогноз принадлежности района к кластеру с использованием модели
    df_neighborhoods_no_ocean_proximity['opp'] = model.predict(log_reg)
    # очистим, изменим формат данных, присоединим к df_neighborhoods_no_ocean_proximity
    df_neighborhoods_no_ocean_proximity['ocean_proximity_predict'] = df_neighborhoods_no_ocean_proximity.opp.astype(int)
    df_neighborhoods_no_ocean_proximity = df_neighborhoods_no_ocean_proximity.drop(['opp'], axis = 1).rename(columns = {'ocean_proximity_predict': cluster})
# очистим итоговый dataframe от мусорных столбцов
df_predict_clusters = df_neighborhoods_no_ocean_proximity.drop(['ocean_proximity'], axis = 1)

In [ ]:
df_predict_clusters.head() # данные о районах с неизвестным кластером + прогноз принадлежности к кластеру на основании логистической регрессии

,id,longitude,latitude,median_house_value,population,<1H_OCEAN,INLAND,ISLAND,NEAR_BAY,NEAR_OCEAN
0,0,-122.23,37.88,452600.0,322.0,1,0,0,0,0
3,3,-122.25,37.85,341300.0,558.0,1,0,0,0,0
6,6,-122.25,37.84,299200.0,1094.0,1,0,0,0,0
7,7,-122.25,37.84,241400.0,1157.0,0,0,0,0,0
9,9,-122.25,37.84,261100.0,1551.0,1,0,0,0,0


In [ ]:
df_predict_clusters.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5013 entries, 0 to 16211
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  5013 non-null   int64  
 1   longitude           5013 non-null   float64
 2   latitude            5013 non-null   float64
 3   median_house_value  5013 non-null   float64
 4   population          5013 non-null   float64
 5   <1H_OCEAN           5013 non-null   int64  
 6   INLAND              5013 non-null   int64  
 7   ISLAND              5013 non-null   int64  
 8   NEAR_BAY            5013 non-null   int64  
 9   NEAR_OCEAN          5013 non-null   int64  
dtypes: float64(4), int64(6)
memory usage: 430.8 KB


### Оценим результаты прогноза

In [ ]:
# построчно для каждого района проссумируем значения в столбцах кластеров: полученное значение покажет, сколько моделей посчитали район своим
df_predict_clusters['count_clusters'] = df_predict_clusters[['<1H_OCEAN', 'INLAND', 'ISLAND', 'NEAR_BAY', 'NEAR_OCEAN']].sum(axis = 1)
df_predict_clusters.groupby('count_clusters').count()

,id,longitude,latitude,median_house_value,population,<1H_OCEAN,INLAND,ISLAND,NEAR_BAY,NEAR_OCEAN
count_clusters,,,,,,,,,,
0,2249,2249,2249,2249,2249,2249,2249,2249,2249,2249
1,2747,2747,2747,2747,2747,2747,2747,2747,2747,2747
2,17,17,17,17,17,17,17,17,17,17


### Итак, обученные нами для каждого кластера модели посчитали "своими" примерно 2804 района (при каждом прогоне кода модели дают различные результаты). Примерно 2200 районов своим не определила не одна модель, зато на порядка 6-20 районов позарились сразу две модели.
### Может быть дело в избыточности предикторов, которые сбивают модели с толку? Давайте построим прогноз на основании одних лишь географических координат.

In [ ]:
# разделим df_neighborhoods на два df: df_neighborhoods_ocean_proximity, где ocean_proximity значим, и df_neighborhoods_no_ocean_proximity, где он None
df_neighborhoods_ocean_proximity = df_neighborhoods[df_neighborhoods['ocean_proximity'].notna()]
df_neighborhoods_no_ocean_proximity = df_neighborhoods[df_neighborhoods['ocean_proximity'].isna()]
# сформируем список кластеров для построения моделей
clusters_list = ['<1H_OCEAN', 'INLAND', 'ISLAND', 'NEAR_BAY', 'NEAR_OCEAN']
# для каждого кластера:
for cluster in clusters_list:
    # построим и обучи модель логистической регрессии
    X, y = df_clusters[['longitude', 'latitude']], df_clusters[cluster]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    model = LogisticRegression()
    model.fit(X_train, y_train)
    # возьмём известные данные с неизвестными кластерами
    log_reg = df_neighborhoods_no_ocean_proximity[['longitude', 'latitude']]
    # и построим прогноз принадлежности района к кластеру с использованием модели
    df_neighborhoods_no_ocean_proximity['opp'] = model.predict(log_reg)
    # очистим, изменим формат данных, присоединим к df_neighborhoods_no_ocean_proximity
    df_neighborhoods_no_ocean_proximity['ocean_proximity_predict'] = df_neighborhoods_no_ocean_proximity.opp.astype(int)
    df_neighborhoods_no_ocean_proximity = df_neighborhoods_no_ocean_proximity.drop(['opp'], axis = 1).rename(columns = {'ocean_proximity_predict': cluster})
# очистим итоговый dataframe от мусорных столбцов
df_predict_clusters_littl = df_neighborhoods_no_ocean_proximity.drop(['ocean_proximity'], axis = 1)

In [ ]:
df_predict_clusters_littl.head()

,id,longitude,latitude,median_house_value,population,<1H_OCEAN,INLAND,ISLAND,NEAR_BAY,NEAR_OCEAN
0,0,-122.23,37.88,452600.0,322.0,0,0,0,0,0
3,3,-122.25,37.85,341300.0,558.0,0,0,0,0,0
6,6,-122.25,37.84,299200.0,1094.0,0,0,0,0,0
7,7,-122.25,37.84,241400.0,1157.0,0,0,0,0,0
9,9,-122.25,37.84,261100.0,1551.0,0,0,0,0,0


In [ ]:
df_predict_clusters_littl['count_clusters'] = df_predict_clusters_littl[['<1H_OCEAN', 'INLAND', 'ISLAND', 'NEAR_BAY', 'NEAR_OCEAN']].sum(axis = 1)
df_predict_clusters_littl.groupby('count_clusters').count()

,id,longitude,latitude,median_house_value,population,<1H_OCEAN,INLAND,ISLAND,NEAR_BAY,NEAR_OCEAN
count_clusters,,,,,,,,,,
0,772,772,772,772,772,772,772,772,772,772
1,3638,3638,3638,3638,3638,3638,3638,3638,3638,3638
2,603,603,603,603,603,603,603,603,603,603


### Н-да... Иногда район своим считают даже **три** модели. И несмотря на то, что распознанных районов стало больше, уменьшение количества предикторов вряд ли хорошая идея.

### **Вывод:** из 5013 записей с неуказанным кластером с использованием модели **логистической регрессии** удаётся классифицировать, в зависимости от используемых предикторов, примерно 2800 или 3600, при этом во втором случае качество прогноза явно ниже. Конечно, можно дополнить данные значениями, полученными на основании прогноза, вновь прогнать через модель, и делать так снова и снова, пока неизвестных кластеров не останется, но основывать прогноз на результатах предыдущего прогноза занятие не очень здоровое.
### Будем считать оптимальным способом решения поставленной задачи использование **линейной регрессии**.